In [ ]:
#import dependencies
import sqlite3
from sqlite3 import Error
from pathlib import Path
from sqlalchemy import create_engine, inspect
import pandas as pd
from json import loads, dumps
from flask import jsonify

In [ ]:
# creaate the database
Path('happiness.sqlite').touch()
#defining the database
conn = sqlite3.connect('happiness.sqlite')
#connect to database
c = conn.cursor()

In [ ]:
# Create a reference to the file. 
database_path = Path("Happiness.sqlite")

In [ ]:
# Create an engine that can talk to the database
engine = create_engine(f"sqlite:///{database_path}")

In [ ]:
# add 2021 happiness table
c.execute('''CREATE TABLE Happiness_2021(Country text Primary Key, Happiness real, GDP_per_capita real,Social_support real,Life_expectancy real,Freedom real,Generosity real,Corruption real,Dystopia real,Contribution_GDP_per_capita real,Contribution_Social_support real,Contribution_Life_expectancy real,Contribution_Freedom real,Contribution_Generosity real,Contribution_Corruption real,Contribution_Dystopia real);''')

In [ ]:
#add metal table and make it reference the happiness table
c.execute('''CREATE TABLE Bands_2021(Country text Primary Key, Metal_bands_per_100000_people, Bands, Population, Foreign Key (Country) References Happiness_2021(Country));''')

In [ ]:
# drop table if necessary
c.execute('''Drop TABLE Happiness_2021;''')

In [ ]:
#check column names for Happiness Table

# Create the inspector and connect it to the engine
inspector = inspect(engine)
# Collect the names of tables within the database
inspector.get_table_names()
# Using the inspector to print the column names within the 'dow' table and its types
columns = inspector.get_columns('Happiness_2021')

In [ ]:
#check column names for Band Table

# Create the inspector and connect it to the engine
inspector = inspect(engine)
# Collect the names of tables within the database
inspector.get_table_names()
# Using the inspector to print the column names within the 'dow' table and its types
columns = inspector.get_columns('Bands_2021')

In [ ]:
# load the happiness data into a Pandas DataFrame
happiness_data = pd.read_csv('2021_Clean.csv')
# write the happiness data to a sqlite table
happiness_data.to_sql('Happiness_2021', conn, if_exists='append', index = False)

In [ ]:
# load the band data into a Pandas DataFrame
happiness_data = pd.read_csv('metal_bands_clean.csv')
# write the happiness data to a sqlite table
happiness_data.to_sql('Bands_2021', conn, if_exists='append', index = False)

In [ ]:
#view the Happiness data
pd.read_sql('''SELECT * FROM Happiness_2021''',conn)

In [ ]:
#view the Band data
pd.read_sql('''SELECT * FROM Bands_2021''',conn)

In [ ]:
#design a query to pull data from the database
combined_df = pd.read_sql('''SELECT b.Country , b.Metal_bands_per_100000_people , b.Bands , b.Population , h.Happiness , 
h.GDP_per_capita , h.Social_support , h.Life_expectancy, h.Freedom , h.Generosity , h.Corruption , h.Dystopia , 
h.Contribution_GDP_per_capita , h.Contribution_Social_support , h.Contribution_Life_expectancy , h.Contribution_Freedom, 
h.Contribution_Generosity , h.Contribution_Corruption , h.Contribution_Dystopia
FROM Bands_2021 b INNER JOIN Happiness_2021 h ON b.Country = h.Country''', conn)
#make country the index
combined_df = combined_df.set_index(list(combined_df)[0])
combined_df.head()

In [ ]:
#convert df to json
result = combined_df.to_json(orient="split")
parsed = loads(result)
dumps(parsed, indent=4)  

In [ ]:
#save the json
combined_df.to_json('combined_json.json')